# 유튜브 크리에이터 정보 웹크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

import time
import pandas as pd


# 리스트 정의
creater_list = []
sub_list = []
video_list = []
view_list = []

interests = ['흥둥이','화니의주방','챔CHAM','짤막한강좌','임한량','양주연',
             '씨디맨','삼순이TV','냥꼬리','꽃피는하루','고기남자','랄로',
             'CorgiTV','사나고','소녀의행성','철구형','오늘의하늘','미스터리튜브',
            '수빙수','haha ha','토모토모','닥터프렌즈','장삐쭈','속삭이는몽자',
            '공돌이 용달','오복이','선바','우주하마','지안씨','뷔브독']

### 정보 수집

In [9]:
for youtuber in interests: 
    
    # youtube 페이지 띄우기
    driver = webdriver.Chrome()
    url ='https://youtube.com/'
    driver.get(url)

    time.sleep(2)


    # 수집할 크리에이터 이름 검색
    driver.find_element_by_css_selector('input#search').send_keys('{}\n'.format(youtuber))

    time.sleep(2)


    # 구독자 수, 동영상 수 정보 수집
    html = bs(driver.page_source, 'html.parser')

    sub = html.select_one('span#subscribers').text
    video_count = html.select_one('span#video-count').text


    # 크리에이터 페이지로 이동
    if html.select('.channel-link.yt-simple-endpoint.style-scope.ytd-channel-renderer') is not None:
        driver.find_element_by_css_selector(".channel-link.yt-simple-endpoint.style-scope.ytd-channel-renderer").click()
    else:
        driver.find_element_by_css_selector("div.container.style-scope.ytd-rich-grid-channel").click()
    time.sleep(2)


    # 크리에이터명 수집
    html = bs(driver.page_source, 'html.parser')

    creater = html.select_one('div#text-container').text.strip()


    # 크리에이터 정보 탭 url 만들고 페이지 종료
    about_url = '{}/about'.format(driver.current_url)
    driver.quit()

    time.sleep(1)


    # 조회수 수집
    driver = webdriver.Chrome()
    driver.get(about_url)
    html = bs(driver.page_source, "html.parser")

    view = html.select("div#right-column > yt-formatted-string.ytd-channel-about-metadata-renderer")
    view_count = view[2].text
    time.sleep(2)
    driver.quit()


    # 리스트에 크리에이터 정보 저장
    e = sub.rfind('만')
    sub_list.append(sub[4:e])

    e1 = sub.rfind('개')
    video_list.append(video_count[4:e1])

    creater_list.append(creater)

    e2 = view_count.rfind('회')
    view_list.append(view_count[4:e2])

print(sub_list, video_list, creater_list, view_list)

['10.2', '10', '10', '9.27', '9.83', '8.33', '8.86', '3.18', '16', '9.98', '96.5', '38.5', '9.26', '290', '87.3', '144', '66.6', '8.92', '94', '104', '64.3', '64.7', '278', '76.4', '57.6', '3.8', '87.9', '117', '12.4', '6.91'] ['49', '437', '132', '316', '333', '235', '4,534', '123', '458', '295', '67', '321', '784', '129', '334', '6,867', '256', '78', '194', '353', '87', '417', '331', '420', '180', '86', '2,049', '1,650', '185', '306'] ['흥둥이', '화니의 주방', '챔CHAM', '짤막한 강좌', '임한량 YHR TV', 'YangJooYeon양주연', '씨디맨', '삼순이TV', '냥꼬리', 'bloomingdays꽃피는하루', '고기남자 MeatMan', '랄로', 'Corgi TV', '사나고 Sanago', '소녀의행성 Girlsplanet', '철구형 (CHULTUBE)', 'Ha Neul오늘의 하늘', '미스터리튜브 MysteryTube', '수빙수tv sooBingsoo', 'haha ha', '토모토모TomoTomo', '닥터프렌즈', '장삐쭈', '속삭이는몽자', '공돌이 용달', '갓오복', 'SUNBA선바', '우주하마', '지안씨JIANSSI', '뷔브독 bwibeu_dog'] ['3,142,380', '15,558,350', '3,850,067', '10,091,187', '44,475,560', '10,707,639', '60,418,433', '4,872,009', '54,324,889', '7,623,296', '82,721,779', '105,858,672', '41,950,703',

### 수집한 정보 저장

In [10]:
# 현재 날짜 정보 만들기
import datetime
from pytz import timezone, utc

KST = timezone('Asia/Seoul')
now = datetime.datetime.utcnow()
Know = utc.localize(now).astimezone(KST).strftime('%Y%m%d')

dic = {'크리에이터':creater_list,
      '구독자':sub_list,
      '동영상':video_list,
      '조회수':view_list}

df = pd.DataFrame(dic)
df.to_csv('{}_크리에이터_정보.csv'.format(Know), encoding='utf_8_sig', index=False)